In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from nltk.corpus import stopwords

In [ ]:
from collections import OrderedDict

In [ ]:
lexicon = pd.read_csv('expandedLexicon.tsv', delimiter='\t')

In [ ]:
df = pd.read_csv('hatespeech.csv')

In [ ]:
df

,class,tweet
0,False,!!! RT @mayasolovely: As a woman you shouldn't...
1,True,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,True,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,True,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,True,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
...,...,...
24778,True,you's a muthaf***in lie &#8220;@LifeAsKing: @2...
24779,False,"you've gone and broke the wrong heart baby, an..."
24780,True,young buck wanna eat!!.. dat nigguh like I ain...
24781,True,youu got wild bitches tellin you lies


In [ ]:
lexicon

,term,score
0,horrible_noun,3.679601
1,disgusting_adj,3.493682
2,moron_noun,3.469677
3,bastard_noun,3.399238
4,stupid_noun,3.323882
...,...,...
8473,recourse_noun,-4.170819
8474,tension_verb,-4.351016
8475,tension_noun,-4.408596
8476,doubt_noun,-5.126174


In [ ]:
text = ''.join(df['tweet'])

In [ ]:
def tokenize_doc(sent,
                 lemma=False,
                 remove_stopwords=False):

    # a simple tokenizer with case folding and an option to use lemmatization
    sent = sent.lower()
    #tokens = sent.split()

    tokens = pd.Series(sent.split())
    if lemma:
        #tokens = [lemmatizer.lemmatize(token) for token in tokens]
        tokens = tokens.apply(lambda x: lemmatizer.lemmatize(x))
    if remove_stopwords:

        #tokens = [token for token in tokens if token not in stopwords.words('english')]
        #tokens_mask = tokens.apply(check_token)
        #tokens = tokens[tokens_mask]


        filter = tokens.isin(stopwords.words('english'))
        tokens = tokens.mask(filter)
        tokens = tokens.dropna()
    return list(tokens)

def basic_text_processing(corpus):
    # This function will go through the corpus and outputs two components
    # w2i: the mapping of a vocabular to in index. This is also our vocabulary
    # docs_in_tokens: list of extracted tokens for each document
    vocab = set()
    docs_in_tokens = []
    for doc in corpus:
      # tokenize_doc(sent = doc)
        tokens = tokenize_doc(doc, lemma=False, remove_stopwords=True)
        vocab.update(set(tokens))
      # vocab is a set of all unique tokens
        docs_in_tokens.append(tokens)
      # docs_in_tokens is a list of lists containing all the tokens in each doc
    vocab = list(vocab)
    # vocab is type list
    vocab.sort()
    # the phrase below is just creating an empty ordered dict
    w2i = OrderedDict()
    for i, word in enumerate(vocab):
      # {word : i}
        w2i[word] = i
    # returns {word : i} and list of all tokens in all the docs. Multiple
    # occurrences allowed in docs_in_tokens but not w2i
    return w2i, docs_in_tokens

In [ ]:
def splitter(term):
  term = term.split('_')
  term = term[0]
  return term

In [ ]:
lexicon['term'] = lexicon.term.apply(splitter)

In [ ]:
lexicon.drop_duplicates(inplace = True)

In [ ]:
w2i, doc_in_tokens = basic_text_processing(df['tweet'].values)

In [ ]:
#w2i has each token labeled by index
#doc_in_token is a list of tokens per document

In [ ]:
w2i = dictionary(w2i)

In [ ]:
w2i = pd.Series(w2i)

In [ ]:
w2i

!                0
!!               1
!!!              2
!!!!             3
!!!!!            4
             ...  
~a           53521
~charlie     53522
~fob         53523
~~           53524
~~ruffled    53525
Length: 53526, dtype: int64

In [ ]:
filter = w2i.index.isin(lexicon.term.values)
w2i_filtered = w2i.where(filter)

In [ ]:
w2i_filtered.dropna(inplace = True)

In [ ]:
w2i_filtered

187          7628.0
1984         7648.0
absent      20152.0
absolute    20153.0
absurd      20156.0
             ...   
wrestle     52868.0
wrong       52887.0
yank        53041.0
yokel       53257.0
zigeuner    53459.0
Length: 1333, dtype: float64

In [ ]:
lexicon_dict = dict(zip(lexicon.term,lexicon.score))

In [ ]:
scores = []
for doc in doc_in_tokens:
  doc_score = 0
  for word in doc:
    value = lexicon_dict.get(word, 0)
    doc_score += value

  scores.append(doc_score)


In [ ]:
scores = np.array(scores)
preds = np.array(scores) >= -1

In [ ]:
np.mean(preds == df['class'].values)

0.7946576282128879

In [ ]:
for thres in range(-20,20,1):
  preds = np.array(all_scores) >= thres/100
